In [2]:
#! /usr/bin/env python

import os
import requests
import soundcloud
import pandas as pd
from datetime import datetime
from time import sleep
import logging
import csv
from pprint import pprint as pp

####################################################################

# Please be nice with this!
CLIENT_ID = '175c043157ffae2c6d5fed16c3d95a4c'
CLIENT_SECRET = '99a51990bd81b6a82c901d4cc6828e46'
MAGIC_CLIENT_ID = 'b45b1aa10f1ac2941910a7f0d10f8e28'

AGGRESSIVE_CLIENT_ID = 'OmTFHKYSMLFqnu2HHucmclAptedxWXkq'
APP_VERSION = '1481046241'

####################################################################

# Sleep time between requests - 2 seconds so as to not overload the server
SLEEP_TIME = 2
inFileName = './data/soundcloud_footwork_tracks_tag.csv'
outFileName = './data/soundcloud_footwork_artists_from_tracks.csv'

####################################################################

In [15]:
def get_client():
    """
    Return a new SoundCloud Client object.
    """
    client = soundcloud.Client(client_id=CLIENT_ID)
    return client

In [16]:
# get client
client = get_client()

In [5]:
# find all sounds of footwork'
response = client.get('/users', q='footwork', limit=page_size,
                    linked_partitioning=1)
#for tracks in tracks.collection:
#    print(track.title)

In [12]:
response_dict = response.fields()
next_href = response_dict['next_href']
results = response_dict['collection']
result_keys = results[0].keys()
data = pd.DataFrame(results)
data = data.drop(columns=['description'])

In [40]:
data['title'][len(data)-1]

'PREMIERE: Scott Diaz & Max Footwork - Theres No Time To Waste'

In [13]:
data.to_csv('footwork_artists.csv')

In [42]:
response_dict = requests.get(next_href).json()
next_href = response_dict['next_href']
results = response_dict['collection']
result_keys = results[0].keys()
new_data = pd.DataFrame(results)
new_data = new_data.drop(columns=['description'])

In [45]:
new_data.to_csv('footwork_tracks.csv', mode='a', header=False)

In [5]:
# Generate list of artists to query from list
# of tracks tagged with footwork
tracks = pd.read_csv(inFileName)

In [6]:
tracks.columns

Index(['Unnamed: 0', 'artwork_url', 'attachments_uri', 'bpm', 'comment_count',
       'commentable', 'created_at', 'download_count', 'download_url',
       'downloadable', 'duration', 'embeddable_by', 'favoritings_count',
       'genre', 'id', 'isrc', 'key_signature', 'kind', 'label_id',
       'label_name', 'last_modified', 'license', 'likes_count',
       'original_content_size', 'original_format', 'permalink',
       'permalink_url', 'playback_count', 'purchase_title', 'purchase_url',
       'release', 'release_day', 'release_month', 'release_year',
       'reposts_count', 'sharing', 'state', 'stream_url', 'streamable',
       'tag_list', 'title', 'track_type', 'uri', 'user', 'user_favorite',
       'user_id', 'user_playback_count', 'video_url', 'waveform_url'],
      dtype='object')

In [111]:
tracks['user'][0]

"{'kind': 'user', 'uri': 'https://api.soundcloud.com/users/1918080', 'avatar_url': 'https://i1.sndcdn.com/avatars-000192885703-d6h82k-large.jpg', 'permalink_url': 'http://soundcloud.com/knitewatch', 'id': 1918080, 'last_modified': '2016/10/14 14:17:09 +0000', 'permalink': 'knitewatch', 'username': 'Knite Watch'}"

In [27]:
response = client.get('/users/1918080')

In [28]:
user = response.fields()

In [49]:
user['description'] = user['description'].replace('\n', ' ').encode("ascii", errors="ignore").decode().strip()

In [50]:
with open(outFileName, 'w') as outFile:  # Just use 'w' mode in 3.x
    outFileWriter = csv.DictWriter(outFile, user.keys())
    outFileWriter.writeheader()
    outFileWriter.writerow(user)

In [7]:
# Group by artist to get aggregate stats so we can sort and query in a sensible order
# sum is the total number of downlaods per artist across all tracks in list
# count is the total number of tracks in the list per artist
grouped = tracks[['user_id', 'download_count']].groupby('user_id')
aggregated = grouped.agg(['sum', 'count'])
# remove the multiindex
aggregated.columns = aggregated.columns.droplevel()
sorted_artists = aggregated.sort_values(by='sum', ascending=False)
sorted_user_id_list = list(sorted_artists.index)

In [11]:
first = sorted_user_id_list.pop(0)

In [23]:
for i, user_id in enumerate(sorted_user_id_list[545:]):
    idx = i+1 + 545
    print(idx, user_id)

546 41039498
547 59719734
548 913524
549 1608359
550 3004858
551 3763300
552 8786014
553 112825
554 5974926
555 113063248
556 165399673
557 4955482
558 1337634
559 3767126
560 13042742
561 8820564
562 96388
563 1178617
564 521590
565 1263595
566 8326167
567 3888223
568 13657793
569 2218409
570 4244960
571 8622061
572 1269098
573 98390
574 14156934
575 73137
576 137959328
577 37308
578 7162814
579 522171
580 63420832
581 2330903
582 476755
583 361595
584 109024670
585 265357
586 50440
587 105131
588 58179
589 480255
590 35672227
591 5801477
592 2127638
593 3358246
594 468991
595 2242000
596 654851
597 2510
598 14547301
599 278266
600 18213490
601 2380798
602 1296572
603 1554564
604 11147389
605 69978550
606 240511
607 3007715
608 82753
609 6201274
610 368769
611 122734833
612 8475137
613 71693736
614 26914213
615 710645
616 4402707
617 5518085
618 1887532
619 37288970
620 9375478
621 171112940
622 17258110
623 674765
624 40661511
625 99540
626 703996
627 236156335
628 177156
629 1582101

1266 885706
1267 44798440
1268 187262212
1269 90508948
1270 6687331
1271 41320334
1272 2403147
1273 52752318
1274 1212425
1275 20574751
1276 7976128
1277 6781695
1278 304029
1279 127525197
1280 83119937
1281 118829
1282 6245218
1283 45267173
1284 283848085
1285 38632
1286 138406
1287 96975705
1288 2098929
1289 3793366
1290 168602360
1291 146336490
1292 984239
1293 93780
1294 1192796
1295 1077174
1296 20317805
1297 71098482
1298 50811638
1299 2852213
1300 39292149
1301 47253041
1302 1572686
1303 10598416
1304 287929
1305 93671916
1306 81074009
1307 13860249
1308 229351253
1309 9825570
1310 15990977
1311 143824194
1312 215875995
1313 15349555
1314 78685388
1315 63706329
1316 178784
1317 8218816
1318 352361
1319 99610507
1320 73725
1321 8251023
1322 11120318
1323 190151257
1324 51446766
1325 259066987
1326 984772
1327 208900856
1328 108694
1329 23090076
1330 5496171
1331 229762431
1332 135112910
1333 76526731
1334 124993312
1335 72406030
1336 175157762
1337 34532415
1338 32430097
1339 577

2259 51436
2260 49145
2261 46146
2262 43061
2263 42921
2264 74937
2265 76950
2266 41257
2267 98520
2268 124213
2269 117224
2270 113429
2271 109865
2272 109369
2273 108003
2274 105036
2275 103748
2276 101219
2277 98398
2278 77608
2279 98006
2280 97790
2281 95982
2282 94349
2283 91573
2284 90888
2285 84191
2286 81507
2287 81070
2288 42901
2289 41044
2290 128090
2291 7880
2292 12270
2293 11604
2294 11352
2295 10305
2296 9962
2297 9837
2298 9833
2299 9660
2300 8229
2301 7711
2302 13716
2303 7677
2304 6877
2305 6060
2306 4870
2307 4804
2308 3426
2309 3390
2310 2160
2311 984
2312 13543
2313 13855
2314 40929
2315 21824
2316 40683
2317 36743
2318 35826
2319 34907
2320 32117
2321 26067
2322 25947
2323 24914
2324 22579
2325 20649
2326 14873
2327 19817
2328 19785
2329 19466
2330 18425
2331 17118
2332 16275
2333 15372
2334 15095
2335 15081
2336 126930
2337 129410
2338 603327
2339 376194
2340 452091
2341 431304
2342 424992
2343 421356
2344 419876
2345 417330
2346 403865
2347 399740
2348 382932
2349

In [3]:
artists = pd.read_csv(outFileName)

In [7]:
artists.columns

Index(['followings_count', 'reposts_count', 'website', 'plan',
       'playlist_count', 'discogs_name', 'city', 'full_name', 'first_name',
       'country', 'public_favorites_count', 'last_modified', 'online',
       'permalink_url', 'myspace_name', 'avatar_url', 'uri', 'last_name',
       'followers_count', 'kind', 'permalink', 'username', 'subscriptions',
       'id', 'website_title', 'track_count'],
      dtype='object')

In [10]:
artists = artists.reindex_axis(sorted(artists.columns), axis=1)

/Users/phillcoletti/anaconda3/envs/scraping/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


In [11]:
artists

,avatar_url,city,country,discogs_name,first_name,followers_count,followings_count,full_name,id,kind,...,plan,playlist_count,public_favorites_count,reposts_count,subscriptions,track_count,uri,username,website,website_title
0,https://i1.sndcdn.com/avatars-000238011936-8ky...,Los Angeles,United States,NaN,t,173123,211,t stewart,1382,user,...,Pro Plus,10,302,127,"[{'product': {'name': 'Pro Unlimited', 'id': '...",130,https://api.soundcloud.com/users/1382,Machinedrum,http://machinedrum.net,machinedrum.net
1,https://i1.sndcdn.com/avatars-000002001516-lau...,Cologne & Osnabrück,NaN,NaN,Trust In Bass,1244,340,Trust In Bass,99309,user,...,Pro Plus,2,90,6,"[{'product': {'name': 'Pro Unlimited', 'id': '...",58,https://api.soundcloud.com/users/99309,Clearcut,NaN,NaN
2,https://i1.sndcdn.com/avatars-000082259560-2fj...,London,NaN,NaN,NaN,23285,51,NaN,125198,user,...,Pro Plus,4,4,25,"[{'product': {'name': 'Pro Unlimited', 'id': '...",69,https://api.soundcloud.com/users/125198,Stray,NaN,NaN
3,https://i1.sndcdn.com/avatars-000313611004-x4a...,Baku,Azerbaijan,NaN,Farhad,1411,43,Farhad Farzali,6243192,user,...,Free,6,330,23,[],64,https://api.soundcloud.com/users/6243192,Azeriff,http://vk.com/azeriffmusic,VK
4,https://i1.sndcdn.com/avatars-000271086025-oua...,Ghetto Paradise,United States,NaN,JUKE BOUNCE WERK,6341,1141,JUKE BOUNCE WERK,16378730,user,...,Pro Plus,26,2222,1719,"[{'product': {'name': 'Pro Unlimited', 'id': '...",377,https://api.soundcloud.com/users/16378730,JUKE BOUNCE WERK,http://www.jukebouncewerk.com/,JBW WEBSITE
5,https://i1.sndcdn.com/avatars-000245018007-tzn...,LOS ANGELES // WORLDWIDE,NaN,NaN,NaN,7182,814,NaN,37227650,user,...,Pro Plus,61,358,29,"[{'product': {'name': 'Pro Unlimited', 'id': '...",229,https://api.soundcloud.com/users/37227650,TAR,http://www.wearetar.com,TAR
6,https://i1.sndcdn.com/avatars-000338012899-xpi...,Los Angeles,United States,NaN,NaN,445163,45,NaN,483960,user,...,Pro Plus,37,1147,43,"[{'product': {'name': 'Pro Unlimited', 'id': '...",421,https://api.soundcloud.com/users/483960,SOULECTION,http://soulection.supply,supply
7,https://i1.sndcdn.com/avatars-000316104749-ve6...,Dallas,United States,NaN,NaN,32131,484,NaN,718195,user,...,Pro,12,533,62,"[{'product': {'name': 'Pro', 'id': 'creator-pr...",27,https://api.soundcloud.com/users/718195,RUN DMT,NaN,NaN
8,https://i1.sndcdn.com/avatars-000341702504-jxa...,Oslo,Norway,NaN,Slick,13528,967,Slick Shoota,2402917,user,...,Pro Plus,10,644,45,"[{'product': {'name': 'Pro Unlimited', 'id': '...",99,https://api.soundcloud.com/users/2402917,Slick Shoota,http://www.slickshoota.com,SLICKSHOOTA.COM
9,https://i1.sndcdn.com/avatars-000053714665-vb6...,Los Angeles,United States,NaN,Alfred,154657,129,Alfred Darlington,1371,user,...,Pro Plus,18,250,33,"[{'product': {'name': 'Pro Unlimited', 'id': '...",317,https://api.soundcloud.com/users/1371,Daedelus,http://www.daedelusmusic.com,Daedelus


In [12]:
artists.to_csv('./data/soundcloud_footwork_artists_from_tracks_v2.csv', index=False)

In [29]:
user.pop('description', 0)
user

{'avatar_url': 'https://i1.sndcdn.com/avatars-000192885703-d6h82k-large.jpg',
 'city': '♥ L.A. ♥',
 'country': 'United States',
 'discogs_name': None,
 'first_name': '',
 'followers_count': 2672,
 'followings_count': 528,
 'full_name': '',
 'id': 1918080,
 'kind': 'user',
 'last_modified': '2016/10/14 14:17:09 +0000',
 'last_name': '',
 'myspace_name': None,
 'online': False,
 'permalink': 'knitewatch',
 'permalink_url': 'http://soundcloud.com/knitewatch',
 'plan': 'Pro Plus',
 'playlist_count': 0,
 'public_favorites_count': 1721,
 'reposts_count': 5,
 'subscriptions': [{'product': {'id': 'creator-pro-unlimited',
    'name': 'Pro Unlimited'}}],
 'track_count': 7,
 'uri': 'https://api.soundcloud.com/users/1918080',
 'username': 'Knite Watch',
 'website': 'https://knitewatch.toneden.io/',
 'website_title': 'Toneden'}

In [30]:
with open('testing.csv', 'w') as testFile:
    testFileWriter = csv.DictWriter(testFile, sorted(user.keys()))
    testFileWriter.writeheader()
    testFileWriter.writerow(user)

In [24]:
user

'I love animals and nature. Oh and I also make beats.\n\nFounder of https://soundcloud.com/2LUSH \n\nSometimes I post here @emtybeats\n\nSnapchat: Knite_Watch\n●▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬●\nBookings/Colabs/Remixes - Knitewatch@gmail.com\n●▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬●\n\nBounce with me! \n\n+++Enjoy+++ \n\n\n'